In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mlp-project-dataset/train_data.csv
/kaggle/input/mlp-project-dataset/test_data.csv


<font size="5"><bold>**Libraries**</bold></font>

In [5]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline, FeatureUnion 
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score

<font size="5"><bold>**Data**</bold></font>

In [6]:
data = pd.read_csv("/kaggle/input/mlp-project-dataset/train_data.csv")
data1 = pd.read_csv('/kaggle/input/mlp-project-dataset/test_data.csv')
data

,HomePage,HomePage_Duration,LandingPage,LandingPage_Duration,ProductDescriptionPage,ProductDescriptionPage_Duration,GoogleMetric:Bounce Rates,GoogleMetric:Exit Rates,GoogleMetric:Page Values,SeasonalPurchase,...,SearchEngine,Zone,Type of Traffic,CustomerType,Gender,Cookies Setting,Education,Marital Status,WeekendPurchase,Made_Purchase
0,0.0,0.000000,0.0,0.0,1.0,0.000000,0.200000,0.200000,0.0,0.0,...,1.0,9.0,3.0,Returning_Visitor,Not Specified,Deny,Not Specified,Other,0.0,False
1,0.0,0.000000,0.0,0.0,2.0,2.666667,0.050000,0.140000,0.0,0.0,...,2.0,2.0,4.0,Returning_Visitor,Female,Deny,Others,Married,0.0,False
2,0.0,0.000000,0.0,0.0,10.0,627.500000,0.020000,0.050000,0.0,0.0,...,3.0,1.0,4.0,Returning_Visitor,Female,ALL,Others,Married,1.0,False
3,0.0,0.000000,0.0,0.0,1.0,0.000000,0.200000,0.200000,0.0,0.4,...,4.0,3.0,3.0,Returning_Visitor,Male,ALL,Diploma,Single,0.0,False
4,1.0,0.000000,0.0,0.0,0.0,0.000000,0.200000,0.200000,0.0,0.0,...,2.0,1.0,5.0,Returning_Visitor,Male,Deny,Diploma,Other,1.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14726,1.0,4.000000,0.0,0.0,39.0,983.138889,0.015385,0.017599,0.0,0.0,...,2.0,6.0,3.0,Returning_Visitor,Female,Deny,NaN,Married,0.0,True
14727,8.0,117.023809,2.0,57.0,11.0,252.892857,0.000000,0.011078,0.0,0.0,...,2.0,2.0,4.0,Returning_Visitor,Not Specified,ALL,Diploma,Married,0.0,True
14728,2.0,75.600000,2.0,652.8,10.0,1143.666667,0.000000,0.023333,0.0,0.0,...,2.0,4.0,2.0,Returning_Visitor,Not Specified,Required,Others,Single,0.0,True
14729,0.0,0.000000,0.0,0.0,6.0,1057.000000,0.000000,0.033333,0.0,0.0,...,4.0,4.0,1.0,Returning_Visitor,Not Specified,Required,Not Specified,Married,0.0,True


<font size="5"><bold>**Data Preprocessing**</bold></font>

In [8]:
train_data_copy = data.copy()
test_data_copy =  data1.copy()

y_train = train_data_copy["Made_Purchase"]

train_data_copy = train_data_copy.drop(["Made_Purchase"],axis=1)

categorical_train = train_data_copy[['Month_SeasonalPurchase', 'CustomerType', 'Gender', 'Cookies Setting',
 'Education', 'Marital Status', 'OS', 'SearchEngine', 'Zone',
 'Type of Traffic', 'WeekendPurchase']]
categorical_test = test_data_copy[['Month_SeasonalPurchase', 'CustomerType', 'Gender', 'Cookies Setting',
 'Education', 'Marital Status', 'OS', 'SearchEngine', 'Zone',
 'Type of Traffic', 'WeekendPurchase']]

numerical_train = train_data_copy[['HomePage' ,'HomePage_Duration', 'LandingPage', 'LandingPage_Duration',
 'ProductDescriptionPage', 'ProductDescriptionPage_Duration',
 'GoogleMetric:Bounce Rates', 'GoogleMetric:Exit Rates',
 'GoogleMetric:Page Values', 'SeasonalPurchase']]
numerical_test = test_data_copy[['HomePage' ,'HomePage_Duration', 'LandingPage', 'LandingPage_Duration',
 'ProductDescriptionPage', 'ProductDescriptionPage_Duration',
 'GoogleMetric:Bounce Rates', 'GoogleMetric:Exit Rates',
 'GoogleMetric:Page Values', 'SeasonalPurchase']]


a= categorical_train.columns.values
print(a)

b= numerical_train.columns.values
print(b)

print(len(a) + len(b))

['Month_SeasonalPurchase' 'CustomerType' 'Gender' 'Cookies Setting'
 'Education' 'Marital Status' 'OS' 'SearchEngine' 'Zone' 'Type of Traffic'
 'WeekendPurchase']
['HomePage' 'HomePage_Duration' 'LandingPage' 'LandingPage_Duration'
 'ProductDescriptionPage' 'ProductDescriptionPage_Duration'
 'GoogleMetric:Bounce Rates' 'GoogleMetric:Exit Rates'
 'GoogleMetric:Page Values' 'SeasonalPurchase']
21


In [9]:
lab_enc = LabelEncoder()
y_train = lab_enc.fit_transform(y_train)

In [10]:
num_pipeline = Pipeline([('imputer', SimpleImputer(strategy = 'mean')),('std_scaler', StandardScaler())])
cat_pipeline = Pipeline([('imputer', SimpleImputer(strategy = 'most_frequent'))])

numerical_train = num_pipeline.fit_transform(numerical_train)
categorical_train = cat_pipeline.fit_transform(categorical_train)
categorical_train = pd.DataFrame(categorical_train, index = None)

df_col=list(categorical_train.columns)

for i in range(len(df_col)):
 categorical_train[df_col[i]] = LabelEncoder().fit_transform(categorical_train[df_col[i]])

numerical_test = num_pipeline.fit_transform(numerical_test)
categorical_test = cat_pipeline.fit_transform(categorical_test)
categorical_test = pd.DataFrame(categorical_test, index = None)

df1_col=list(categorical_test.columns)

for i in range(len(df1_col)):
 categorical_test[df1_col[i]] = LabelEncoder().fit_transform(categorical_test[df1_col[i]])

In [11]:
X_train = np.concatenate((numerical_train, categorical_train), axis = 1)
X_test  = np.concatenate((numerical_test, categorical_test), axis = 1)
X_train.shape

(14731, 21)

<font size="5"><bold>**Decision Tree Classifier**</bold></font>

In [ ]:
from sklearn.tree import DecisionTreeClassifier 

params = {'criterion' : ['gini','entropy'], 'max_depth' : range(1,10), 'min_samples_split' : range(1,10), 'min_samples_leaf' : range(1,5)}

DTC = DecisionTreeClassifier(random_state = 0)
model  = GridSearchCV(DTC, params, cv = 5, return_train_score = True, error_score=0)
model.fit(X_train,y_train)

In [ ]:
model.best_params_

In [13]:
from sklearn.tree import DecisionTreeClassifier 

DTCfinal =  DecisionTreeClassifier(random_state=0, criterion = 'gini', max_depth = 1, min_samples_leaf = 1, min_samples_split = 2)
DTCfinal.fit(X_train,y_train)
pred = DTCfinal.predict(X_train)
print(accuracy_score(y_train,pred))
print(f1_score(y_train,pred))

0.6645848890095717
0.42392444910807975


In [ ]:
predictions = DTCfinal.predict(X_test)
submission = predictions.astype(np.bool)
df = pd.DataFrame({"id" : np.arange(X_test.shape[0]), "Made_Purchase" : submission})
df.to_csv("DecisionTree.csv", index = None)

<font size = "3">The comparison of performance of different models with hyperparameter tuning can be viewed at: </font>https://colab.research.google.com/drive/1HZNEMEOP0f84Nb4WRrb6I5eJ-aQWYMzY?usp=sharing 

<font size="5"><bold>**KNeighbors Classifier**</bold></font>

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

parameters  = {'n_neighbors' : np.arange(1,11) }

kneigh = KNeighborsClassifier()

model  = GridSearchCV(kneigh, parameters, cv = 5, return_train_score = True)
model.fit(X_train,y_train)

In [ ]:
model.best_params_

In [ ]:
kfinal = KNeighborsClassifier(n_neighbors = 2)
kfinal.fit(X_train, y_train)
predictions = kfinal.predict(X_test)
submission = predictions.astype(np.bool)
df = pd.DataFrame({"id" : np.arange(X_test.shape[0]), "Made_Purchase" : submission})
df.to_csv("KNeighbors.csv", index = None)

<font size="5"><bold>**Gaussian Naive Bayes**</bold></font>

In [18]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)
predictions = gnb.predict(X_test)
submission = predictions.astype(np.bool)
df = pd.DataFrame({"id" : np.arange(X_test.shape[0]), "Made_Purchase" : submission})
df.to_csv("GaussianNB.csv", index = None)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """


<font size="5"><bold>**Logistic Regression**</bold></font>

In [ ]:
from sklearn.linear_model import LogisticRegression
  
params = {'C' : np.logspace(-3, 3, 10), 'penalty' : ['l1', 'l2'], 'max_iter':[100,1000]}

LR = LogisticRegression(random_state=0)

model  = GridSearchCV(LR, params, cv = 5, return_train_score = True, error_score=0)
model.fit(X_train,y_train)

In [ ]:
model.best_params_

In [17]:
LRfinal = LogisticRegression(random_state=0, C= 0.1,penalty='l2', max_iter=100)
LRfinal.fit(X_train,y_train)
predictions = LRfinal.predict(X_test)
submission = predictions.astype(np.bool)
df = pd.DataFrame({"id" : np.arange(X_test.shape[0]), "Made_Purchase" : submission})
df.to_csv("Logistic.csv", index = None)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


<font size="5"><bold>**Perceptron**</bold></font>

In [ ]:
from sklearn.linear_model import Perceptron 

perp = Perceptron(random_state=0)

params = {'alpha' : np.logspace(-3, 3, 10), 'max_iter':[100,1000]}

model  = GridSearchCV(perp, params, cv = 5, return_train_score = True, error_score=0)
model.fit(X_train,y_train)

In [ ]:
model.best_params_

In [ ]:
perpfinal =  Perceptron(random_state=0, alpha = 0.001 , max_iter=100 )

perpfinal.fit(X_train,y_train)
predictions = perpfinal.predict(X_test)
submission = predictions.astype(np.bool)
df = pd.DataFrame({"id" : np.arange(X_test.shape[0]), "Made_Purchase" : submission})
df.to_csv("Perceptron.csv", index = None)


<font size="5"><bold>**Random Forest Classifier**</bold></font>

In [ ]:
from sklearn.ensemble import RandomForestClassifier 

params = {'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

RFC = RandomForestClassifier(random_state=0)
model  = RandomizedSearchCV(RFC, params, cv = 5, return_train_score = True, error_score=0)
model.fit(X_train,y_train)

In [ ]:
model.best_params_

In [ ]:
RFCfinal =  RandomForestClassifier(random_state=0, bootstrap = True, max_depth = 10, max_features = 'sqrt', min_samples_leaf = 4, min_samples_split =2 , n_estimators = 2000 )

RFCfinal.fit(X_train,y_train)
predictions = RFCfinal.predict(X_test)
submission = predictions.astype(np.bool)
df = pd.DataFrame({"id" : np.arange(X_test.shape[0]), "Made_Purchase" : submission})
df.to_csv("RandomForest.csv", index = None)

<font size="5"><bold>**Gradient Boosting Classifier**</bold></font>

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

params = {
    "n_estimators":[5,50,250,500],
    "max_depth":[1,3,5,7,9],
    "learning_rate":[0.01,0.1,1,10,100]
}
GBC = GradientBoostingClassifier(random_state=0)
model  = RandomizedSearchCV(GBC, params, cv = 5, return_train_score = True, error_score=0)
model.fit(X_train,y_train)

In [ ]:
model.best_params_

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

GBCfinal =  GradientBoostingClassifier(random_state=0, max_depth = 1, learning_rate = 0.01, n_estimators = 500)

GBCfinal.fit(X_train,y_train)
predictions = GBCfinal.predict(X_test)
submission = predictions.astype(np.bool)
df = pd.DataFrame({"id" : np.arange(X_test.shape[0]), "Made_Purchase" : submission})
df.to_csv("GradientBoost.csv", index = None)

<font size="5"><bold>**Ada Boost Classifier**</bold></font>

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

params = {'n_estimators' :[10, 50, 100, 200, 500] , 'learning_rate' : [0.0001, 0.01, 0.1, 1.0, 1.1, 1.2]}

ABC = AdaBoostClassifier(random_state=0)
model1  = RandomizedSearchCV(ABC, params, cv = 5, return_train_score = True, error_score=0)
model1.fit(X_train,y_train)
model1.best_params_

In [ ]:
ABCfinal =  AdaBoostClassifier(random_state=0, learning_rate = 0.0001, n_estimators = 10)

ABCfinal.fit(X_train,y_train)
predictions = ABCfinal.predict(X_test)
submission = predictions.astype(np.bool)
df = pd.DataFrame({"id" : np.arange(X_test.shape[0]), "Made_Purchase" : submission})
df.to_csv("AdaBoost.csv", index = None)

<font size="5"><bold>**XGB Classifier**</bold></font>

In [ ]:
from xgboost import XGBClassifier

params = {'n_estimators' :[10, 50, 100, 200, 500] , 'learning_rate' : [0.0001, 0.01, 0.1, 1.0, 1.1, 1.2], 'max_depth' : [1,3,5,7,9], 'max_leaves' : [0,1,3,5,7,9]}

xgb = XGBClassifier(random_state=0)
model1  = GridSearchCV(xgb, params, cv = 5, return_train_score = True, error_score=0)
model1.fit(X_train,y_train)
model1.best_params_

In [ ]:
from xgboost import XGBClassifier
xgbfinal =  XGBClassifier(random_state=0, learning_rate = 0.0001, n_estimators = 10, max_leaves = 0, max_depth = 1)

xgbfinal.fit(X_train,y_train)
predictions = xgbfinal.predict(X_test)
submission = predictions.astype(np.bool)
df = pd.DataFrame({"id" : np.arange(X_test.shape[0]), "Made_Purchase" : submission})
df.to_csv("XGB.csv", index = None)

<font size="5"><bold>**Overall Scores**</bold></font>

In [21]:
table = pd.read_csv("/kaggle/input/overall-scores/overall_scores.csv")
table


,Classifier Name,Train Score,Test Score
0,KNeighborsClassifier,0.69,0.57
1,GaussianNB,0.64,0.62
2,LogisticRegression,0.66,0.63
3,Perceptron,0.57,0.55
4,RandomForestClassifier,0.70,0.63
5,DecisionTreeClassifier,0.66,0.64
6,GradientBoostingClassifier,0.67,0.64
7,AdaBoostClassifier,0.66,0.64
8,XGBClassifier,0.66,0.64
